In [1]:
import datetime 
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from datetime import date
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
import requests

In [2]:
#create a driver
driver = webdriver.Chrome(ChromeDriverManager().install())

# link to extract
driver.get("https://finance.yahoo.com/")

# set a scroll limit
scroll_limit = 160

# set count, to count how many times we scroll
count = 0

#assign a dummy to new height
new_height=0

# break if we scroll less than scroll limit defined
while True and count < scroll_limit:

    # give 15 second to make sure all page load
    time.sleep(15)
    
    # to get the height of our document
    height = driver.execute_script("return document.documentElement.scrollHeight")
    
    # if new height = height obtain, we should break the while loop
    if new_height == height:
        break
        
    # now new height is the height now
    new_height = height
    # execute script to scrol until end of height
    driver.execute_script("window.scrollTo(0, " + str(height) + ");")
    
    # count our scroll limit
    count+=1
time.sleep(10)
    
page = driver.page_source
driver.close()

C:\Users\CHIN\AppData\Local\Temp/ipykernel_14424/3168819238.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
# parse our page to html parser using beautiful soup
soup = BeautifulSoup(page,"html.parser")
# obtain the header table
tabl = soup.find_all("li" , {"class" : "js-stream-content Pos(r)"})

In [35]:
header = []
category =[]
link =[]
val=0
for i in tabl:
    val+=1
    header_value = i.findAll('div')[0].findAll('div')[0].findAll('h3')[0].text
    link_value = i.findAll('a',href=True)[0]['href']
    if len(i.findAll('div')[0].findAll('div')[0])>=3:
        category_value =i.findAll('div')[0].findAll('div')[0].findAll('div')[3].text
    else:
        category_value ='NA'
    header.append(header_value)
    category.append(category_value)
    link.append(link_value)

In [41]:
#process the data
df= pd.DataFrame({'header': header,'category': category,'link':link})
# remove advertisement
remove_ads = df[df['category']!='NA'].copy()

In [42]:
#fix link infront the link we extract
fix_link = 'https://finance.yahoo.com'

# row by row to extract the link and get our article content
for i in range(remove_ads.shape[0]):
    header = remove_ads.header[i:i+1].values[0]
    url = fix_link+remove_ads.link[i:i+1].values[0]
    # get page content
    page = requests.get(url, allow_redirects=True)
    if (page.status_code == 200):
        page_content = page.content
        soup = BeautifulSoup(page_content,"html.parser")
        remove_ads.loc[remove_ads['header']==header,'article_content'] = soup.findAll('div',{'class':'caas-body'})[0].text

In [46]:
remove_ads.head()

,header,category,link,article_content
0,This company has wiped out more investor wealt...,Business,/m/2748db21-e293-3769-8d57-c2983cf614e4/this-c...,NaN
2,"McDonald's unveils first automated location, s...",Business,/news/mcdonalds-unveils-first-automated-locati...,McDonald’s opened its first automated restaura...
3,Elon Musk Warns Against Margin Debt on Risk of...,Business,/news/elon-musk-warns-against-margin-215641054...,(Bloomberg) -- Billionaire Elon Musk is warnin...
4,Bill Gates Hit By Housing Market Swoon? Billio...,Business,/news/bill-gates-hit-housing-market-192520787....,More evidence for the housing market downturn ...
5,'Not the time to get greedy': Home flippers ge...,Business,/news/not-time-greedy-home-flippers-110000999....,'Not the time to get greedy': Home flippers ge...


In [72]:
remove_ads.to_csv(r'C:\Users\CHIN\Downloads\medium_dev\Extract US index news\Output\news'+str(date.today())+'.csv',index=False)